# KerasTuner demo

`KerasTuner` is a general purpose hyperparameter tuning library. It is integrated with Keras worflows but it is not limited to them. It can be used to tune `scikit-learn` models.

## Define the search space

In the following code example, we deinfe a Keras model with two `Dense` layers. We want to tune the number of units in the first `Dense` layer. We just define an integer hyperparameter with `hp.Int('units', min_value=32, max_value=512, step=32)` whose range is from `32` to `512`inclusive. When sampling from it, the minimum step for walking through the interval is `32`.

In [3]:
import keras
from keras import layers

def build_model_with_two_dense_layers(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # define the hyperparameter
            units=hp.Int("unit", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model
    

Let us test if the model was built successfully

In [7]:
import keras_tuner

build_model_with_two_dense_layers(keras_tuner.HyperParameters())

<Sequential name=sequential_2, built=False>

There are many other types of hyperparameters as well. We can define multiple hyperparameters in the function. In the following code, we tune whether to use a `Dropout` layer with `hp.Boolean()`, tine which activation function to use with `hp.Choice`, tune the learning rate of the optimizer with `hp.Float()`.

In [9]:
def build_model_with_two_dense_layers_and_dropout(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            # Tune number of units
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            # Tune the activation function to use
            activation=hp.Choice("activation", ["relu", "tanh"]),
        )
    )
    # Tune whether to use dropout.
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    # Define the optimizer learning rate as a hyperparameter
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

build_model_with_two_dense_layers_and_dropout(keras_tuner.HyperParameters())

<Sequential name=sequential_3, built=False>

As shown below, the hyperparameters are actual values. In fact, they are just functions returning actual values. For example, `hp.Int()` returns an `int` value. Therefore, you can put them into variables, `for` loops, or `if` conditions. 

In [10]:
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value=32, max_value=512, step=32))

32


The hyperparameters can be defined in advance and keep the Keras model code in a separate function as shown below

In [12]:
def call_existing_code(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(units=units, activation=activation))
    if dropout:
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values
    model = call_existing_code(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model

build_model(keras_tuner.HyperParameters())

<Sequential name=sequential_4, built=False>